# Team 1 CS 105 Final Project

Our project works with....Our dataset is...

In total, the downloaded dataset has 50,000 reviews split between 25,000 train and test sets. There is also an even split of positive and negative reviews (25,000 total for each) among the train and test sets. A negative label correspondes to an Imdb raiting of 1-4 and a positive label corresponds to an Imdb rating of 7-10. For our project, we plan to build our classifier using a smaller subset of 5,000 total reviews. We decided that a stratified sample between positive and negative reviews would give us the most balanced labeled set to work with. Since we aim to work with 5,000 reviews total, we will split our sample into 2,500 positive and negative reviews, which we will pull from the train set.

In [6]:
import os
import shutil
import numpy as np

# Code to select 2500 negative reviews from train set
sub_dir_neg = 'E:/CS105-Final-Project/aclImdb/train/neg'
files_neg = os.listdir(sub_dir_neg)

random_2500_neg = np.random.choice(files_neg, 2500, replace=False)

for x in random_2500_neg:
    shutil.copy(os.path.join(sub_dir_neg, x),'E:/CS105-Final-Project/Project Datasets/neg')

In [ ]:
# Code to select 2500 positive reviews from train set
sub_dir_pos = 'E:/CS105-Final-Project/aclImdb/train/pos'
files_pos = os.listdir(sub_dir_pos)

random_2500_pos = np.random.choice(files_pos, 2500, replace=False)

for x in random_2500_pos:
    shutil.copy(os.path.join(sub_dir_pos, x),'E:/CS105-Final-Project/Project Datasets/pos')